In [1]:
import pandas as pd

from tqdm import tqdm
from src.data_utils import clean_string, to_tokenizer
#from src import data_utils

d:\DS\YandexPrac\practice\prac\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Загрузка, очистка, токенизация датасета

In [2]:
with open('data/tweets.txt', 'r', encoding='utf-8') as f:
    texts = f.readlines()

#df = pd.DataFrame({'text': texts})
df = pd.DataFrame({'text': texts[:1000]})

In [4]:
df['clean_text'] = [clean_string(text) for text in tqdm(df['text'])] #, desc="Очистка текстов")
df = df[df['clean_text'].str.len() > 0].reset_index(drop=True)

100%|██████████| 1000/1000 [00:00<00:00, 3965.87it/s]


In [6]:
encode = to_tokenizer(df)